In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
train_df_original = pd.read_csv('TRAIN_FINAL.csv')
test_df = pd.read_csv('Test_v1.csv')

In [5]:
positive_dataset = train_df[train_df_original['TARGET']==0]

In [7]:
negative_dataset = train_df[train_df_original['TARGET']==1]

In [13]:
sample_positive = positive_dataset.sample(n=1763)

In [19]:
train_df = pd.concat([sample_positive,negative_dataset])

In [20]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3526 entries, 7755 to 45606
Columns: 370 entries, col_0 to TARGET
dtypes: float64(111), int64(259)
memory usage: 10.0 MB


In [21]:
train_df = train_df.sample(frac = 1)

In [23]:
X = train_df.drop(['TARGET'],axis=1)

y = train_df['TARGET']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42,stratify = y)

In [24]:
import lightgbm as lgb 

lgb_params = {"objective" : "binary", "metric" : ['auc'],
              "num_leaves" : 8, "learning_rate" : 0.05, 
              "is_unbalance": True,
              "bagging_fraction" : 0.5, "feature_fraction" : 0.8, "bagging_frequency" : 9,
               }
    
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_val = lgb.Dataset(X_test, label=y_test)
model = lgb.train(lgb_params, lgb_train, 2000, valid_sets=[lgb_val], verbose_eval=100)

[100]	valid_0's auc: 0.834008
[200]	valid_0's auc: 0.831583
[300]	valid_0's auc: 0.82881
[400]	valid_0's auc: 0.82699
[500]	valid_0's auc: 0.824193
[600]	valid_0's auc: 0.824065
[700]	valid_0's auc: 0.821616
[800]	valid_0's auc: 0.819973
[900]	valid_0's auc: 0.817425
[1000]	valid_0's auc: 0.815539
[1100]	valid_0's auc: 0.813912
[1200]	valid_0's auc: 0.811152
[1300]	valid_0's auc: 0.809844
[1400]	valid_0's auc: 0.80812
[1500]	valid_0's auc: 0.807051
[1600]	valid_0's auc: 0.805783
[1700]	valid_0's auc: 0.804112
[1800]	valid_0's auc: 0.802726
[1900]	valid_0's auc: 0.800518
[2000]	valid_0's auc: 0.799803


In [25]:
ind = test_df['index']
test = test_df.drop(['index'],axis=1)

In [26]:
pred_val = model.predict(test,num_iteration=model.best_iteration)

y_out = [1 if i > 0.5 else 0 for i in pred_val]

In [30]:
dfdict={}

dfdict["index"]=ind
dfdict["TARGET"]=y_out

df=pd.DataFrame(dfdict)
df.to_csv("solution.csv",index=False,columns=["index","TARGET"])